In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python
!pip install --upgrade nltk --target ./python

In [ ]:
import sys
sys.path.append(r"./python")

import os
import json
import traceback
import urllib.parse
import boto3
from datetime import datetime
import time
from python.smart_search import SmartSearchQA

In [ ]:
#根据时间情况修改index和language值

index =  "data_index_test"
language = "english"
EMBEDDING_ENDPOINT_NAME = "huggingface-inference-eb"


port = 443
bulk_size = 10000000


sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
host = host[8:-1]
region = boto3.Session().region_name # e.g. cn-north-1
print('host:',host)
print('region:',region)

# retrieve secret manager value by key using boto3                                             
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')


In [ ]:
sys.path.append(r"./python")
if language == "english":
    import nltk
    nltk.download('punkt')

search_qa = SmartSearchQA()
search_qa.init_cfg(index,
                 username,
                 password,
                 host,
                 port,
                 EMBEDDING_ENDPOINT_NAME,
                 region,
                 language=language
                 )

local_path = "../docs/"

chunk_size = 1500

files = os.listdir(local_path)
i = 0
for file in files:
    local_file = local_path+file
    now1 = datetime.now()#begin time
    loaded_files = search_qa.init_knowledge_vector(local_file,bulk_size,chunk_size)
    now2 = datetime.now()#endtime
    i += 1
    print(i," Complete the import of the following documents:", str(loaded_files)," File import takes time:",now2-now1 )

